<a href="https://colab.research.google.com/github/tphlabs/Lab4_FH_teach/blob/main/FH_lab_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4. Frank-Hertz experiment

Igor Gitelman, 2024, v0.3

<p>Here we need to import measured data from the measuring program and then plot it. As you already know we need to plot the Frank-Hertz curve (characteristic curve).</p>

<p>We start by importing general modules that we use here, and in most of the lab experiments.</p>

In [3]:
import numpy as np # math functions
import matplotlib.pyplot as plt # for plotting figures and setting their properties
import pandas as pd # handling data structures (loaded from files)
from pathlib import Path # convenient file-system handling
from scipy.signal import find_peaks as find_peaks # Find peaks inside a signal based on peak properties.

<p> In the future we will use other modules too.</p>
<p>Now we use the <b>pandas</b> module, we called him <b>pd</b>, to import the data from file "FH1.csv", where the ending ".csv" stand for <b>comma separated value</b>. Pay attention to the actual separator, it may be different depending on the data type.</p>

In [17]:
# Place the path to your data here.
# When using Anaconda locally UPLOAD your data to the Anaconda environment.
URL = 'FH1.csv'

fh1=pd.read_csv(URL, sep='\t',header=5)
fh1.head()

,Va(V)_1,Ia(E-12 A)_1,T(c)_1,Va(V)_2,Ia(E-12 A)_2,T(c)_2
0,0.0,0.20,169.93,0.0,0.18,169.94
1,0.1,0.25,169.94,0.1,0.49,169.95
2,0.2,0.19,169.94,0.2,0.52,169.95
3,0.3,0.16,169.94,0.3,0.24,169.95
4,0.4,0.19,169.95,0.4,36.30,169.94


About the read_csv function, you can read [here](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) or you can google it.</p>
'\t' indicates a tab separator between columns.</p>
The function starts to read from line 5 that contains the column names, because of "header=5".

Now, take the names of the columns, and create relevant variables

In [11]:
Va1 = np.array(fh1['Va(V)_1']) # accelerating voltage vector
I1 = np.array(fh1['Ia(E-12 A)_1']) # Current vector
T1 = np.array(fh1['T(c)_1']) #temperature vector

Set the heater current, retard voltage depedning in your experiment parameters 

In [ ]:
Heater_Current=        #V
V_retard=              #V

one quick way to find the peaks of the characteristic curve is by using find_peaks function

In [12]:
peaks, _ = find_peaks(I1, prominence=10 ,width=3)
peaks

array([ 61, 108, 156, 205], dtype=int32)

peaks variable will contain the indexes in which the peaks occur

<h2>Data plot</h2>

<p>Now we use <b>matplotlib.pyplot</b> we called him <b>plt</b> in order to plot the data.</p>

In [33]:
plt.figure(dpi=200) # Define new figure, and set "Dots per Inch" or "dpi"|

<Figure size 1280x960 with 0 Axes>

<Figure size 1280x960 with 0 Axes>

And the distances between pick voltages

## Frank-Hertz runs at 150 °C and 160 °C

The next cells expect the CSV files `150c.csv` and `160c.csv` to be placed next to this notebook. Each file should contain at least the accelerating voltage (`Va`, `Voltage`, etc.) and the anode current (`Ia`, `Current`, etc.). The helper functions below take care of loading the data, plotting the characteristic curve, and annotating the peak voltages automatically.

In [ ]:
def _locate_header_row(csv_path: Path) -> int:
    """Return how many lines must be skipped to reach the header row."""
    with csv_path.open("r", encoding="utf-8", errors="ignore") as handle:
        for idx, line in enumerate(handle):
            lower = line.lower()
            if ("va" in lower or "volt" in lower) and ("ia" in lower or "curr" in lower):
                return idx
    return 0  # assume header is already in the first row


def _select_column(columns, keywords):
    for keyword in keywords:
        for col in columns:
            if keyword in col.lower():
                return col
    raise ValueError(f"None of the columns match {keywords}. Found: {columns}")


def load_voltage_current(csv_path: str):
    """Load a CSV file and return voltage/current numpy arrays with metadata."""
    csv_path = Path(csv_path)
    if not csv_path.exists():
        raise FileNotFoundError(f"Could not find {csv_path.resolve()} - please upload the file.")

    header_row = _locate_header_row(csv_path)
    df = pd.read_csv(csv_path, sep=None, engine="python", skiprows=header_row)
    df.columns = [col.strip() for col in df.columns]

    voltage_col = _select_column(df.columns, ("va", "volt"))
    current_col = _select_column(df.columns, ("ia", "curr"))

    df = df.dropna(subset=[voltage_col, current_col])
    voltage = df[voltage_col].astype(float).to_numpy()
    current = df[current_col].astype(float).to_numpy()

    metadata = {
        "voltage_column": voltage_col,
        "current_column": current_col,
        "header_row": header_row,
    }
    return voltage, current, metadata


def plot_run_with_peaks(csv_path: str, label: str, prominence: float = 5.0, width: float = 3.0):
    """Plot the characteristic curve and annotate the peak voltages."""
    voltage, current, metadata = load_voltage_current(csv_path)
    peaks, properties = find_peaks(current, prominence=prominence, width=width)
    peak_voltages = voltage[peaks]

    fig, ax = plt.subplots(figsize=(8, 5), dpi=150)
    ax.plot(voltage, current, color="tab:blue", lw=1.2)
    ax.scatter(voltage[peaks], current[peaks], color="tab:red", zorder=3)

    for idx in peaks:
        ax.annotate(
            f"{voltage[idx]:.2f} V",
            (voltage[idx], current[idx]),
            textcoords="offset points",
            xytext=(6, 8),
            ha="left",
            fontsize=9,
            color="tab:red",
        )

    ax.set_title(f"Frank-Hertz curve @ {label}")
    ax.set_xlabel(f"Accelerating voltage ({metadata['voltage_column']})")
    ax.set_ylabel(f"Anode current ({metadata['current_column']})")
    ax.grid(True, which="both", alpha=0.3)

    if len(peaks) == 0:
        print(f"{label}: no peaks found with prominence={prominence}, width={width}.")
    else:
        voltages_fmt = ", ".join(f"{v:.2f} V" for v in peak_voltages)
        print(f"{label}: peak voltages -> {voltages_fmt}")

    return {
        "label": label,
        "csv_path": str(csv_path),
        "peak_indices": peaks,
        "peak_voltages": peak_voltages,
        "prominence": prominence,
        "width": width,
    }

In [ ]:
DATA_FILES = {
    "150 °C": "150c.csv",
    "160 °C": "160c.csv",
}

# Feel free to tweak the peak detection parameters per run if needed.
PEAK_PARAMETERS = {
    "150 °C": {"prominence": 8.0, "width": 3.0},
    "160 °C": {"prominence": 8.0, "width": 3.0},
}

peak_summaries = []
for label, path in DATA_FILES.items():
    params = {"prominence": 5.0, "width": 3.0}
    params.update(PEAK_PARAMETERS.get(label, {}))
    try:
        summary = plot_run_with_peaks(path, label, **params)
        peak_summaries.append(summary)
    except FileNotFoundError as exc:
        print(exc)

if peak_summaries:
    summary_table = pd.DataFrame(
        [
            {
                "Run": item["label"],
                "CSV": item["csv_path"],
                "Peak voltages (V)": ", ".join(f"{v:.2f}" for v in item["peak_voltages"]),
            }
            for item in peak_summaries
        ]
    )
    display(summary_table)
